# ToDo:
- merge app_info with labels.
- merge app_info with events info, drop dublicates.
- write device_id to app data

done in here.

In [1]:
%run preamble.ipynb

The preamble handles a few imports and defines the following functions:

printLoss(y_tr,p_tr,y_te,p_te,loss=log_loss)
createDataSet(predictions, group_encoder, device_ids)
getBestPrediction(data,var='device_id')



# Merge `app_info` with `labels`

In [2]:
unique = pd.read_csv("files/app_unique_info.csv")

In [6]:
app_info = pd.read_csv("files/finalSets/apps.csv"
                       , usecols=["event_id","app_id","is_active"]) # has only active apps

In [7]:
app_info.head(2)

,event_id,app_id,is_active
0,2,5927333115845830913,1
1,2,-5720078949152207372,0


In [10]:
app_info.duplicated().value_counts()

False    31800268
dtype: int64

In [11]:
app_info = app_info.merge(unique, on="app_id", how="left")

In [12]:
app_info.head(2)

,event_id,app_id,is_active,nCats,UniqueLabel
0,2,5927333115845830913,1,4.0,cat_Family
1,2,-5720078949152207372,0,4.0,cat_Productivity


In [13]:
app_info.to_csv("files/finalSets/apps_wiCats.csv", index=False)

# Merge `app_info` with `evts_hasApp` data

In [14]:
evts_hasApp = pd.read_csv("files/finalSets/evts_withApp_ActInst.csv",index_col=0)

In [16]:
gtr=pd.read_csv("files/gender_age_train.csv", usecols=["device_id"])
gte=pd.read_csv("files/gender_age_test.csv", usecols=["device_id"])

In [21]:
evts_hasApp.device_id.isin(gtr.device_id).value_counts()

False    889770
True     556375
Name: device_id, dtype: int64

In [22]:
evts_hasApp.device_id.isin(gte.device_id).value_counts()

True     889770
False    556375
Name: device_id, dtype: int64

In [25]:
consistent(evts_hasApp,evts_hasApp,gtr.device_id, gte.device_id)

len uniques ids: d1 58462  d2:  58462
sum:  116924

intersection of d1 and d2:  58462

Unique train in d1 and d2:  23290
Unique test in d1 and d2:  35172

intersec of train ids in both sets:  23290
intersec of test ids in both sets:  35172


In [15]:
app_info.event_id.isin(evts_hasApp.event_id.unique()).value_counts()

True    31800268
Name: event_id, dtype: int64

In [26]:
evts_hasApp.head(2)

,event_id,device_id,longitude,latitude,day,time,hour,usageDay,isTrain,group,nActiveApps,nInstallApps
0,2,-6401643145415154744,103.65,30.97,Sun,00:54:12,0,3,0,none,6,19
1,6,1476664663289716375,0.00,0.00,Sun,00:27:21,0,3,1,M22-,15,38


In [27]:
evts_hasApp = evts_hasApp.drop(["day", "time","latitude","longitude"], axis=1)

In [29]:
app_info = app_info.merge(evts_hasApp, on="event_id", how="left")

In [30]:
app_info.head(10)

,event_id,app_id,is_active,nCats,UniqueLabel,device_id,hour,usageDay,isTrain,group,nActiveApps,nInstallApps
0,2,5927333115845830913,1,4.0,cat_Family,-6401643145415154744,0,3,0,none,6,19
1,2,-5720078949152207372,0,4.0,cat_Productivity,-6401643145415154744,0,3,0,none,6,19
2,2,-1633887856876571208,0,5.0,cat_Education,-6401643145415154744,0,3,0,none,6,19
3,2,-653184325010919369,1,3.0,cat_Esoteric,-6401643145415154744,0,3,0,none,6,19
4,2,8693964245073640147,1,4.0,cat_Family,-6401643145415154744,0,3,0,none,6,19
5,2,4775896950989639373,1,7.0,cat_Game_Skill,-6401643145415154744,0,3,0,none,6,19
6,2,-8022267440849930066,0,6.0,cat_Game_Skill,-6401643145415154744,0,3,0,none,6,19
7,2,9112463267739110219,0,4.0,cat_Shopping,-6401643145415154744,0,3,0,none,6,19
8,2,-3725672010020973973,0,1.0,cat_Game_Sports,-6401643145415154744,0,3,0,none,6,19
9,2,7167114343576723123,1,8.0,cat_Other,-6401643145415154744,0,3,0,none,6,19


In [20]:
app_info.to_csv("files/finalSets/apps_wiEvts.csv", index=False)

# Merge `app_info` with `device` info

In [2]:
app_info = pd.read_csv("files/finalSets/apps_wiEvts.csv")

In [3]:
app_info.head(2)

,event_id,app_id,nCats,UniqueLabel,device_id,longitude,latitude,hour,usageDay,isTrain,group,nActiveApps,nInstallApps
0,2,5927333115845830913,4,cat_Family,-6401643145415154744,103.65,30.97,0,3,0,none,6,19
1,2,-653184325010919369,3,cat_Esoteric,-6401643145415154744,103.65,30.97,0,3,0,none,6,19


In [32]:
device_info = pd.read_csv("files/ga_hasEvts.csv",usecols=["device_id","phone_brand","device_model","nEvts"])

In [33]:
device_info.head(2)

,device_id,phone_brand,device_model,nEvts
0,-8260683887967679142,小米,MI 2,1
1,7477216237379271436,华为,荣耀6 plus,7


In [34]:
app_info = app_info.merge(device_info, on="device_id", how="left")

In [35]:
app_info.head(2)

,event_id,app_id,is_active,nCats,UniqueLabel,device_id,hour,usageDay,isTrain,group,nActiveApps,nInstallApps,phone_brand,device_model,nEvts
0,2,5927333115845830913,1,4.0,cat_Family,-6401643145415154744,0,3,0,none,6,19,三星,Galaxy Grand Prime,73
1,2,-5720078949152207372,0,4.0,cat_Productivity,-6401643145415154744,0,3,0,none,6,19,三星,Galaxy Grand Prime,73


In [36]:
consistent(app_info,app_info,gtr.device_id,gte.device_id)

len uniques ids: d1 58462  d2:  58462
sum:  116924

intersection of d1 and d2:  58462

Unique train in d1 and d2:  23290
Unique test in d1 and d2:  35172

intersec of train ids in both sets:  23290
intersec of test ids in both sets:  35172


In [37]:
app_info.to_csv("files/finalSets/apps_full.csv", index=False)

# Merge `evts` with `phone_brand` and `device_model` 

In [39]:
evts = pd.read_csv("files/finalSets/evts_noApp.csv")
ga = pd.read_csv("files/ga_hasEvts.csv")

In [40]:
evts.head(1)

,event_id,device_id,longitude,latitude,day,time,hour,usageDay,isTrain,group
0,1,29182687948017175,121.38,31.24,Sun,00:55:25,0,3,1,M39+


In [41]:
evts.device_id.isin(ga.device_id.values).value_counts()

True    1714272
Name: device_id, dtype: int64

In [42]:
evts.merge(ga[["device_id","phone_brand","device_model","nEvts"]], on="device_id", how="left")\
.to_csv("files/finalSets/evts_noApp_phone.csv", index=False)

In [43]:
consistent(evts,evts,gtr.device_id, gte.device_id)

len uniques ids: d1 28239  d2:  28239
sum:  56478

intersection of d1 and d2:  28239

Unique train in d1 and d2:  11278
Unique test in d1 and d2:  16961

intersec of train ids in both sets:  11278
intersec of test ids in both sets:  16961
